<a href="https://colab.research.google.com/github/VEENA9/Leaf-/blob/master/Arimac_by_me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#lib

In [3]:
!pip3 install Pillow
!pip3 install pytesseract
!pip3 install pdf2image
!sudo apt-get install tesseract-ocr
!apt-get install poppler-utils 
!pip install python-poppler

  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=72c9e3177b5811d745d3b4d701624578f6e28c203961f7ef694a3e30f2955357
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 

#Filtred DF

In [56]:
# get data from image
def save_df(df_img,last_blok):

  table_Cv = df_img
  # image data to DF
  table_Cv_df = pd.DataFrame([x.split('\t') for x in table_Cv.split('\n')])
  # image data to DF
  # add headera same as 1st row
  table_Cv_df1 = table_Cv_df.rename(columns=table_Cv_df.iloc[0])
  # remove 1st row
  table_Cv_df1  = table_Cv_df1[1:]

  # filter DF (empty text, left and top 0, remove "e":line)
  Df_filter1 = table_Cv_df1.loc[(table_Cv_df1.text != ' ') & (table_Cv_df1.left != '0') & 
                              (table_Cv_df1.top != '0')& (table_Cv_df1.text != 'e')& (table_Cv_df1.text != "")& (table_Cv_df1.conf != "0")]

  Df_filter1 = Df_filter1.rename(columns={'block_num': 'block_n'})
  add_no = pd.to_numeric(Df_filter1["block_n"]) +last_blok
  Df_filter1['block_num'] = add_no
  Df_filter1 = Df_filter1.dropna()
  next_first_blok = Df_filter1["block_num"].iloc[0]
  # print(Df_filter1) 
  return Df_filter1,next_first_blok                          

#return head an name

In [75]:
def find_head(dff):
  Df_filter = dff  
  # Find key heading
  key_word = ["CONTACT","Education","Project","EXPERIENCE","Referees","Skill","About","Award","Professional","INTEREST",
              "ACTIVITIES","AWARDS","PUBLICATION","Activity","Hobbies","Profile","work","QUALIFICATION"]
  awoid_key = ["www","@","1","2","3","4","5","6","7","8","9","0",":","=","_",",","&"]
  # get guessed heading  
  pattern = '|'.join(key_word)
  guessed_heading_df =Df_filter[Df_filter['text'].str.contains(pattern,na=False, case=False)]
  # get list of guessed head lock num
  # print(guessed_heading_df["text"].tolist())
  block_num_h_guessd = guessed_heading_df["block_num"].values.tolist()
  # print(guessed_heading_df)

  # awoid symbols
  awoid_pattern = '|'.join(awoid_key)
  df_awoid_symbol =Df_filter[Df_filter['text'].str.contains(awoid_pattern,na=False, case=False)] 
  # print(df_awoid_symbol["block_num"].tolist())
  # print(df_awoid_symbol["text"])
  df_awoid = Df_filter[~Df_filter['block_num'].isin(df_awoid_symbol["block_num"].values.tolist())]
  
  # print(df_awoid["text"].tolist())
  # get the guessed heading block number
    # get original (awoid symbol) block list
  block_num_all = df_awoid["block_num"].values.tolist()
  # print(block_num_all)
  # find raws based same block_num 
  full_heading_df = df_awoid[df_awoid['block_num'].isin(block_num_h_guessd)]
  # print(full_heading_df["block_num"].values.tolist())
  # block number from original df
  same_block_no=full_heading_df['block_num'].values.tolist()
  # print(same_block_no)
  # 1 or 2 blocks items (we assume heading must on 1 or 2 words) (get 1 or 2 block word from originl df)
  must_head_blok_list = [item for item, count in collections.Counter(same_block_no).items() if (count <= 3)]
  # print(must_head_blok_list)
  # find df of must head 
  one_or_two_blok_heading1 = Df_filter[Df_filter['block_num'].isin(must_head_blok_list)]
  # print(one_or_two_blok_heading1)

  # ///////////// -----|^ find gessed all heading

  
  # key waigth////// in one_or_two_blok_heading 
  numaric_w = pd.to_numeric(df_awoid['width'])
  numaric_h = pd.to_numeric(df_awoid['height'])
  df_waigth = numaric_w * numaric_h
  # charector count
  D_text_count = df_awoid['text'].str.len()
  df_key_waight_all =df_waigth / D_text_count
  # print(df_key_waight_all)
  # //////////
  
  

  # /////////////// get gussed heading waight and find minimum number of k_waight 
  numaric_w_h = pd.to_numeric(one_or_two_blok_heading1['width'])
  numaric_h_h = pd.to_numeric(one_or_two_blok_heading1['height'])
  df_waigth_head = numaric_w_h * numaric_h_h
  # print(df_waigth_all.values.tolist())
  D_text_count_head = one_or_two_blok_heading1['text'].str.len()
  df_key_waight_head = df_waigth_head / D_text_count_head
  avg_key_waight = df_key_waight_head.min(skipna=True)
  # print(avg_key_waight)
  # ///////////////

  # print(Df_filter[Df_filter['key waigth'] > 25860 ])
  Df_waight_filter = df_key_waight_all[df_key_waight_all >= avg_key_waight] 
  # print(Df_waight_filter)

  Df_waight_filter_text = (pd.concat([Df_waight_filter, df_awoid.reindex(Df_waight_filter.index)], axis=1)).dropna()
  # print(Df_waight_filter_text)
  
  # 1 or 2 or 3 line contain heading : validate
  block_num_h1 = Df_waight_filter_text["block_num"].values.tolist()
  # print(block_num_h1)

  # must_head_blok_list = [item for item, count in collections.Counter(block_num_h1).items() if (count <= 3)]
  # print(block_num_h1)
  full_heading_df1 = Df_filter[Df_filter['block_num'].isin(block_num_h1)]
  fewline_head = full_heading_df1["block_num"].tolist()
  must_head_blok_list = [item for item, count in collections.Counter(fewline_head).items() if (count <= 3)]
  df_headind = Df_filter[Df_filter['block_num'].isin(must_head_blok_list)]



  # NAME
  # pattern = '|'.join(key_word)
  # df_headind_n =df_headind[~df_headind['text'].str.contains(pattern,na=False, case=False)]  
  # numaric_w_n = pd.to_numeric(df_headind_n['width'])
  # numaric_h_n = pd.to_numeric(df_headind_n['height'])
  # df_waigth_n = numaric_w_n * numaric_h_n
  # # charector count
  # D_text_count = df_headind_n['text'].str.len()
  # df_key_waight_all =df_waigth_n / D_text_count  
  # name_index = df_key_waight_all.idxmax()  
  # # print(name_index)
  # block_num = df_headind_n.loc[name_index].block_num
  # # print(block_num)
  # # find raws based same block_num 
  # df2 =df_headind_n.loc[df_headind_n["block_num"] == block_num]
  # # get text of same raw and mearg as a string.
  # ln = df2['text'].values
  # print(' '.join(ln))



  return df_headind

#find head (heading, no heading)

In [64]:
def details(Df_filter,Df_head):
  word = "EXPERIENCE"
  head_index = Df_head[Df_head['text'].str.contains(word, na=False, case=False)]
  # print(head_index)
  if head_index.empty == True:
    print(word," is not in the Heading list.")
    no_heading(Df_filter,Df_head,word)
  else:
    print(word, "is in the Heading list.")
    heading(Df_filter,Df_head,word,head_index)

# heading = TRUE

In [65]:
# heading = TRUE
def heading(Df_filter,Df_head,word,head_index):
    
  # get Details(df) between two heading
  start_index = head_index.index.values[0] 
  # print(start_index)
  df_head_index_list = Df_head.index.values.tolist()
  end_index = df_head_index_list[(df_head_index_list.index(start_index) + 1) % len(df_head_index_list)] -1
  # print(end_index)
  df_data_word = Df_filter.loc[start_index+1 : end_index]
  # print(df_data_word)

  # get number of block count , between two heading(ovvoru blocklayum periya k_waightku uriya word.a eduthu, athu Key_wordku
  # samana irukkanu paarthu = enda athodaye stop pannanum, illana adutha head vara vaasikanum)
  # get block numbers
  blok_tuple = (df_data_word["block_num"].drop_duplicates())
  # print(df_data_word[df_data_word["block_num"].isin([17])])
  blok_len = len(blok_tuple)
  # print(blok_len)
  b = 0
  while (b < blok_len):
    blok_num=blok_tuple.iloc[b]
    a_blok_text_set = df_data_word[df_data_word["block_num"].isin([blok_num])]
    numaric_w_w = pd.to_numeric(a_blok_text_set['width'])
    numaric_h_w = pd.to_numeric(a_blok_text_set['height'])
    df_waigth_word = numaric_w_w * numaric_h_w
    D_text_count_head = a_blok_text_set['text'].str.len()
    df_key_waight_word = df_waigth_word / D_text_count_head
    max_key_waight = df_key_waight_word.idxmax()
    max_w_word_a_blok = df_data_word["text"].loc[[max_key_waight]]
    
    key_word = ["Education","Project","Referees","Skill","Award","Professional","INTEREST",
                "ACTIVITIES","PUBLICATION","Activity","Hobbies"]

    pattern = '|'.join(key_word)
    df_if_in_word = max_w_word_a_blok.str.contains(pattern,na=False, case=False).values[0]
    # print(df_if_in_word)
    if df_if_in_word == True:
      end_index = max_w_word_a_blok  
      break  
    b += 1

  df_data_word_finl = Df_filter.loc[start_index+1 : end_index]
  ln = df_data_word_finl['text'].values
  details_text = '\n'.join(' '.join(ln).split('. '))
  print(details_text)

# Heading Fales

In [8]:
# Heading Fales 
# word.a thedi eduthu, athila periya k_waight irukratha select panni
# athukku aduthatha ulla head.a thedi eduththu, idaila ullatha df aakkanum, next ,Heading True.la ulla pola seiyanum.

def no_heading(Df_filter,Df_head,word):

  head_index_tuple = Df_filter[Df_filter['text'].str.contains(word, na=False, case=False)]
  # print(head_index_tuple)

  if head_index_tuple.empty:
    print("No word in the cv")    

  else:
    numaric_w_w = pd.to_numeric(head_index_tuple['width'])
    numaric_h_w = pd.to_numeric(head_index_tuple['height'])
    df_waigth_word = numaric_w_w * numaric_h_w
    D_text_count_head = head_index_tuple['text'].str.len()
    df_key_waight_word = df_waigth_word / D_text_count_head
    # print(df_key_waight_word)
    max_key_waight = df_key_waight_word.idxmax()
    # print(max_key_waight)
    max_w_word_a_blok = Df_filter["text"].loc[[max_key_waight]]

    big_word_index = max_w_word_a_blok.index[0]
    # print(big_word_index)
    # find next head in Df_head
      # find last head index
    # print(Df_head["text"].tolist())
    head_index = Df_head.index.tolist() 
    last_head_index = Df_head.iloc[[-1]].index[0]
    print(last_head_index , big_word_index )

    if last_head_index <= big_word_index:
      df_data_word_bigword = Df_filter.loc[big_word_index+1 :]  
      print("word in last heading")

    else: 
      print("word in center area")
      next_head_bigword = [i for i in head_index if i > big_word_index][0]
      # print(head_index)
      print(big_word_index+1 , next_head_bigword-1 )
      df_data_word_bigword = Df_filter.loc[big_word_index+1 : next_head_bigword-1]
      # print(df_data_word_bigword)

    ln = df_data_word_bigword['text'].values
    details_text = '\n'.join(' '.join(ln).split('. '))
    print(details_text)  

#DF by OCR - main

In [76]:
import collections
import numpy as np
import pandas as pd
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract


def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path('/content/drive/MyDrive/Example/Dinuka_Rumesh_SE.pdf')


def ocr_core(file):
    df_text = pytesseract.image_to_data(file)
    return df_text

def head_find(Df_filter1):
    dff1 = Df_filter1
    Df_head_e = pd.DataFrame([]) 
    df_head1=find_head(dff1) 
    Df_head_e = Df_head_e.append(df_head1)
    return Df_head_e


def print_pages(pdf_file):
    images = pdf_to_img(pdf_file)
    Df_filter_e = pd.DataFrame([])
    # Df_head = pd.DataFrame([])
    for pg, img in enumerate(images):
        df_img = ocr_core(img)
        # dff = save_df(df_img)
        if Df_filter_e.empty == True:
          df_fill1,next_first_blok=save_df(df_img,0) 
          Df_filter_e = Df_filter_e.append(df_fill1,ignore_index=True)
        else:
          last_blok = Df_filter_e["block_num"].iloc[-1]+1
          blok_add_no = last_blok - next_first_blok
          df_fill1,next_first_blok=save_df(df_img,blok_add_no) 
          Df_filter_e = Df_filter_e.append(df_fill1,ignore_index=True)
        
    
    Df_head = head_find(Df_filter_e)
    details(Df_filter_e,Df_head)
    print(Df_head)
    # print(Df_filter_e)

    # Df_head.to_csv (r'/content/head.csv', index = False, header=True)
    # Df_filter_e.to_csv (r'/content/data1.csv', index = False, header=True) 
       
print_pages('sample.pdf')

EXPERIENCE  is not in the Heading list.
252 76
word in center area
77 251
in Software Development and Testing
Furthermore, | am adept at handling multiple tasks on a daily basis competently and at working well under pressure
A key strength is communication; building strong relationships with people in order to deliver the best results
UNIVERSITY OF MORATUWA: FACULTY OF INFORMATION TECHNOLOGY Reading for B.Sc
(Hons.) Degree in Information Technology
(Expected 2019) SIVALI CENTRAL COLLAGE, RATHNAPURA
G.C.E Advanced Level - Physical Science Stream (2014) Results: Chemistry - B, Combined Mathematics - B, Physics - B SIVALI CENTRAL COLLAGE, RATHNAPURA G.C.E Ordinary Level (2010) Results: 7 A's 2 B's MOBITEL ENGINEERING DIVISION | 2018 September - 2019 February Intern Software Engineering Experience: Engineering Asset Management Portal It is a resource management system which is developed with the intention of monitoring the resources issued by the seven divisions at Mobitel
Technologies Use

# find Details

In [ ]:
a, b = print_pages('sample.pdf')
Df_head = a
Df_filter = b
# print(df_head)
# print(Df_filter)

#moving avg

In [ ]:
# df_last_values = one_or_two_blok_heading["key waigth"]
# print(df_last_values)
# Data_point_m= df_last_values.rolling(2, min_periods=1).mean()
# Data_point_m_D = Data_point_m.squeeze().tolist()
# round_to_whole = [round(num) for num in Data_point_m_D]
# print(round_to_whole)
# may_reduse=((sum(round_to_whole) / len(round_to_whole)) *0.25)
# moving_avg = ((sum(round_to_whole) / len(round_to_whole)))
# may_lower = moving_avg - may_reduse
# if (may_lower < min(df_last_values)):
#   avg_key_waight = min(df_last_values)
# else:
#   avg_key_waight = min(round_to_whole)  

#org

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import collections

# Path of the pdf
PDF_file = "/content/drive/MyDrive/Example/Alagaretnam Elakyaveena.pdf"

'''
Part #1 : Converting PDF to images
'''

# Store all the pages of the PDF in a variable
pages = convert_from_path(PDF_file, 500)

# Counter to store images of each page of PDF to image
image_counter = 1

# Iterate through all the pages stored above
for page in pages:

	# Declaring filename for each page of PDF as JPG
	# For each page, filename will be:
	# PDF page 1 -> page_1.jpg
	# PDF page 2 -> page_2.jpg
	# PDF page 3 -> page_3.jpg
	# ....
	# PDF page n -> page_n.jpg
	filename = "page_"+str(image_counter)+".jpg"
	
	# Save the image of the page in system
	page.save(filename, 'JPEG')

	# Increment the counter to update filename
	image_counter = image_counter + 1

'''
Part #2 - Recognizing text from the images using OCR
'''
	# 3
# Variable to get count of total number of pages
filelimit = image_counter-1

# Creating a text file to write the output
outfile = "out_text.txt"

# Open the file in append mode so that
# All contents of all images are added to the same file
f = open(outfile, "a")

# Iterate from 1 to total number of pages
for i in range(1, filelimit + 1):

	# Set filename to recognize text from
	# Again, these files will be:
	# page_1.jpg
	# page_2.jpg
	# ....
	# page_n.jpg
	filename = "page_"+str(i)+".jpg"
		
	# Recognize the text as string in image using pytesserct
	text = str(((pytesseract.image_to_string(Image.open(filename)))))

	# The recognized text is stored in variable text
	# Any string processing may be applied on text
	# Here, basic formatting has been done:
	# In many PDFs, at line ending, if a word can't
	# be written fully, a 'hyphen' is added.
	# The rest of the word is written in the next line
	# Eg: This is a sample text this word here GeeksF-
	# orGeeks is half on first line, remaining on next.
	# To remove this, we replace every '-\n' to ''.
	text = text.replace('-\n', '')	

	# Finally, write the processed text to the file.
	f.write(text)

# Close the file after writing all the text.
f.close()


#name

In [ ]:
# get data from image
table_Cv = pytesseract.image_to_data(Image.open(filename))
# image data to DF
table_Cv_df = pd.DataFrame([x.split('\t') for x in table_Cv.split('\n')])
# print(table_Cv_df)
# table_Cv = pytesseract.image_to_data(Image.open(filename))
# image data to DF
# table_Cv_df = pd.DataFrame([x.split('\t') for x in table_Cv.split('\n')])
# add headera same as 1st row
table_Cv_df1 = table_Cv_df.rename(columns=table_Cv_df.iloc[0])
# remove 1st row
table_Cv_df1  = table_Cv_df1[1:]

# filter DF (empty text, left and top 0, remove "e":line)
Df_filter = table_Cv_df1.loc[(table_Cv_df1.text != ' ') & (table_Cv_df1.left != '0') & 
                             (table_Cv_df1.top != '0')& (table_Cv_df1.text != 'e')& (table_Cv_df1.text != "")]
name = (Df_filter.width) 
Df_filter.to_csv (r'/content/amilD.csv', index = False, header=True)
# get high widht size as it is a NAME
big_word =(max(Df_filter.width.astype(float)))
# get the big size word block number
block_num = Df_filter["block_num"].iloc[Df_filter["width"].astype(float).argmax()]
# find raws based same block_num 
df2 =Df_filter.loc[Df_filter["block_num"] == block_num]
# get text of same raw and mearg as a string.
ln = df2['text'].values
print(' '.join(ln))

AWARDS & ACHIEVEMENTS


#word

In [ ]:
#find the word, get that row tuple
word = "Experience"
# word = "Education"
# find word tuple
df_tuple_word=Df_filter[Df_filter['text'].str.contains(word, na=False, case=False)]
# index of the word in original DF and next index of filter df////////////////
# array of filtred df index
place_num =Df_filter.index.values
# index of original df
word_index = df_tuple_word.index.values[0]
# original index number on array
x = np.where(place_num == word_index)
# next to filterd df index of word in array
index_new = x[0][0] + 1
# index of original df
next_indx = place_num[index_new]
# block number of next to word bloc number
next_blok_num = Df_filter.block_num[next_indx]
# print(next_blok_num)
# if heading and details in same block (check next raw is in same with word block number)
if (int(next_blok_num) == int(df_tuple_word['block_num'])):
  df3_b_num = int(df_tuple_word['block_num']) 
  
else:
  # sum one to find next raw num (experiance details in start from next line of experiance)
  df3_b_num = int(df_tuple_word['block_num']) + 1
# get df that: experiance data in same block number
df_same_bloc_num =df_tuple_word.loc[df_tuple_word["block_num"] == str(df3_b_num)]
print(df_same_bloc_num)
# get totel line of expariance
# print(df_same_bloc_num.par_num.astype(float))
try:
  max_line_num = max(df_same_bloc_num.par_num.astype(float))
except:
  max_line_num = 1

# loop for get line by line basd on par_num
p_n = 1
print(max_line_num)
while (p_n <= max_line_num):
  # creat df : for one line (filter based same par_num)
  df_par_num =df_same_bloc_num.loc[df_same_bloc_num["par_num"] == str(p_n)]
  # add word of same par_num
  ln2 = df_par_num['text'].values
  word_set = ' '.join(ln2)
  p_n =p_n+1;  
print(word_set)  

IndexError: ignored

#findHead Algo

#save df1

In [ ]:
table_Cv = pytesseract.image_to_data(Image.open(filename))
# print(table_Cv.shape)
table_Cv_df = pd.DataFrame([x.split('\t') for x in table_Cv.split('\n')])
print(table_Cv_df)
table_Cv_df.to_csv (r'/content/export_dataframe.csv', index = False, header=True)